In [1]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset, DatasetDict, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Trainer
import pandas as pd
import torch
import evaluate
import os

BATCH_SIZE = 12
NUM_EPOCHS = 8
N_GEN = 50
base_checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(base_checkpoint)

#ds = load_dataset("ColumbiaNLP/FLUTE").shuffle(seed=42)
df = pd.read_csv("complete_dataset.csv").fillna("")
ds = Dataset.from_pandas(df).shuffle(seed=42)
folds = StratifiedKFold(n_splits=10, shuffle=False)
splits = folds.split(ds, ds['label'])
indexes = [t for t in splits]

In [2]:
from flute_dream import add_combined_cols

ds = ds.map(add_combined_cols)

Map:   0%|          | 0/7534 [00:00<?, ? examples/s]

In [3]:
def preprocess_dataset_s1(examples):
    model_inputs = tokenizer(examples['premise_hypothesis'])
    labels = tokenizer(examples['label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s2(examples):
    model_inputs = tokenizer(examples['premise_hypothesis_system_2'])
    labels = tokenizer(examples['type_label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s31(examples):
    model_inputs = tokenizer(examples['premise_hypothesis_emotion'])
    labels = tokenizer(examples['label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s32(examples):
    model_inputs = tokenizer(examples['premise_hypothesis_motivation'])
    labels = tokenizer(examples['label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s33(examples):
    model_inputs = tokenizer(examples['premise_hypothesis_consequence'])
    labels = tokenizer(examples['label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s34(examples):
    model_inputs = tokenizer(examples['premise_hypothesis_rot'])
    labels = tokenizer(examples['label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s35(examples):
    model_inputs = tokenizer(examples['premise_hypothesis_all_dims'])
    labels = tokenizer(examples['label_explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s41(examples):
    model_inputs = tokenizer(examples['premise_hypothesis'])
    labels = tokenizer(examples['label'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s42(examples):
    model_inputs = tokenizer(examples['premise_hypothesis_label'])
    labels = tokenizer(examples['explanation'])
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def preprocess_dataset_s7(examples):
    lbl_exp = [ex[ex.find("Explanation: "):] + ex[:ex.find("Explanation: ")] for ex in examples['label_explanation']]
    model_inputs = tokenizer(examples['premise_hypothesis'])
    labels = tokenizer(lbl_exp)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def get_path(name):
    return f"{name}/{os.listdir(name)[0]}"

def get_prem_hyp(s):
    ind = s.find("Hypothesis: ")
    prem = s[len("Premise: "):ind]
    hyp = s[ind + len("Hypothesis: "):]
    return [prem, hyp]

'''Class encapsulating the two steps of System 4 (Classify, then Explain)'''
class DREAM_FLUTE_System4 :
    def __init__(self, tokenizer = None, model_s41_path = None, model_s42_path = None) -> None:
        self.tokenizer = tokenizer if tokenizer is not None else AutoTokenizer.from_pretrained("t5-small")
        self.model_s41 = AutoModelForSeq2SeqLM.from_pretrained(model_s41_path) if model_s41_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S4-Classify")
        self.model_s42 = AutoModelForSeq2SeqLM.from_pretrained(model_s42_path) if model_s42_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S4-Explain")

    '''Expected input for function : "Premise: ... . Hypothesis: ... . Is there a contradiction or entailment between the premise and hypothesis ?" 
    Or list of strings in this format'''
    def prediction_pipeline(self, inputs) :
        if isinstance(inputs, str) :
            tok_input = self.tokenizer(inputs, return_tensors='pt').input_ids
            output_model_1 = self.model_s41.generate(tok_input, max_new_tokens=100)
            decoded_output_model_1 = "Label: " + self.tokenizer.decode(output_model_1[0], skip_special_tokens=True)
            intermediate_input = inputs[:inputs.find("Is there a contradiction or entailment between the premise and hypothesis ?")] + decoded_output_model_1 + ". What is the explanation of the label associated to the premise and the hypothesis ?"
            tok_intermediate_input = self.tokenizer(intermediate_input, return_tensors='pt').input_ids
            output_model_2 = self.model_s42.generate(tok_intermediate_input, max_new_tokens=100)

            return decoded_output_model_1 + ". Explanation: " + self.tokenizer.decode(output_model_2[0], skip_special_tokens=True)
        
        elif isinstance(inputs, list) and all(isinstance(input, str) for input in inputs) :
            predictions = []
            for input in inputs :
                predictions.append(self.prediction_pipeline(input))
            
            return predictions
        
        else :
            raise TypeError('Inputs should be either a list of two strings or a list of lists of two strings')
        

'''Ensemble class that loads all models from HuggingFace (or from the device if a path to the model is indicated) 
and implements the ensembling algorithm given in the DREAM-FLUTE paper'''
class DREAM_FLUTE_Ensemble :
    def __init__(self, tokenizer_path = None, s1_path = None, s2_path = None,
                 s3_emo_path = None, s3_mot_path = None, s3_cons_path = None,
                 s3_rot_path = None, s3_alldims_path = None, s4_clas_path = None, 
                 s4_exp_path = None, dream_path = None) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path) if tokenizer_path is not None else AutoTokenizer.from_pretrained("t5-small")
        self.model_s1 = AutoModelForSeq2SeqLM.from_pretrained(s1_path) if s1_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S1")
        self.model_s2 = AutoModelForSeq2SeqLM.from_pretrained(s2_path) if s2_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S2")
        self.model_s3_emo = AutoModelForSeq2SeqLM.from_pretrained(s3_emo_path) if s3_emo_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S3-Emotion")
        self.model_s3_mot = AutoModelForSeq2SeqLM.from_pretrained(s3_mot_path) if s3_mot_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S3-Motivation")
        self.model_s3_cons = AutoModelForSeq2SeqLM.from_pretrained(s3_cons_path) if s3_cons_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S3-Consequence")
        self.model_s3_rot = AutoModelForSeq2SeqLM.from_pretrained(s3_rot_path) if s3_rot_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S3-ROT")
        self.model_s3_alldims = AutoModelForSeq2SeqLM.from_pretrained(s3_alldims_path) if s3_alldims_path is not None else AutoModelForSeq2SeqLM.from_pretrained("YoanBOUTE/DREAM-FLUTE-S3-AllDims")
        self.model_s4 = DREAM_FLUTE_System4(self.tokenizer, s4_clas_path, s4_exp_path)
        self.model_dream = AutoModelForSeq2SeqLM.from_pretrained(dream_path) if dream_path is not None else AutoModelForSeq2SeqLM.from_pretrained("RicoBorra/DREAM-t5-small")
    
    '''Tokenizes the input, then feeds it to the given model, and decodes the output to have a string as result.
    This method is callable for all models except System 4 (Use the method defined in the class of System 4)'''
    def _prediction_pipeline(self, input : str, model) -> str :
        tokenized_input = self.tokenizer(input, return_tensors='pt').input_ids
        model_output = model.generate(tokenized_input, max_new_tokens=100)
        decoded_output = self.tokenizer.decode(model_output[0], skip_special_tokens=True)
        return decoded_output
    
    '''Preprocesses the input for each model, then feeds it to the pipeline.
    Returns a dictionary of all models' predictions.'''
    def _get_all_predictions(self, input : list) :
        prem, hyp = input 
        prem = prem.strip()
        hyp = hyp.strip()
        if not prem.endswith('.') :
            prem += '.'
        if not hyp.endswith('.') :
            hyp += '.' 

        predictions = dict()

        input_1 = f"Premise: {prem} Hypothesis: {hyp} Is there a contradiction or entailment between the premise and hypothesis ?"
        predictions['S1'] = self._prediction_pipeline(input_1, self.model_s1)

        input_2 = f"Premise: {prem} Hypothesis: {hyp} What is the type of figurative language involved? Is there a contradiction or entailment between the premise and hypothesis ?"
        predictions['S2'] = self._prediction_pipeline(input_2, self.model_s2)

        # DREAM elaborations for system 3
        input_dream_prem = f"[SITUATION] {prem} [QUERY] "
        input_dream_hyp = f"[SITUATION] {hyp} [QUERY] "
        prem_elaborations = {key : self._prediction_pipeline(input_dream_prem + key, self.model_dream) for key in ['emotion', 'motivation', 'consequence', 'rot']}
        for key, elab in prem_elaborations.items() :
            elab = elab.strip()
            if not elab.endswith('.') :
                prem_elaborations[key] += '.' 
        hyp_elaborations = {key : self._prediction_pipeline(input_dream_hyp + key, self.model_dream) for key in ['emotion', 'motivation', 'consequence', 'rot']}
        for key, elab in hyp_elaborations.items() :
            elab = elab.strip()
            if not elab.endswith('.') :
                hyp_elaborations[key] += '.' 

        input_3_emo = f"Premise: {prem} [Emotion] {prem_elaborations['emotion']} Hypothesis: {hyp} [Emotion] {hyp_elaborations['emotion']} Is there a contradiction or entailment between the premise and hypothesis ?"
        predictions['S3-emo'] = self._prediction_pipeline(input_3_emo, self.model_s3_emo)

        input_3_mot = f"Premise: {prem} [Motivation] {prem_elaborations['motivation']} Hypothesis: {hyp} [Motivation] {hyp_elaborations['motivation']} Is there a contradiction or entailment between the premise and hypothesis ?"
        predictions['S3-mot'] = self._prediction_pipeline(input_3_mot, self.model_s3_mot)

        input_3_cons = f"Premise: {prem} [Consequence] {prem_elaborations['consequence']} Hypothesis: {hyp} [Consequence] {hyp_elaborations['consequence']} Is there a contradiction or entailment between the premise and hypothesis ?"
        predictions['S3-cons'] = self._prediction_pipeline(input_3_cons, self.model_s3_cons)

        input_3_rot = f"Premise: {prem} [Rot] {prem_elaborations['rot']} Hypothesis: {hyp} [Rot] {hyp_elaborations['rot']} Is there a contradiction or entailment between the premise and hypothesis ?"
        predictions['S3-rot'] = self._prediction_pipeline(input_3_rot, self.model_s3_rot)

        input_3_all = f"Premise: {prem} "
        for key, elab in prem_elaborations.items() :
            input_3_all += f"[{key.capitalize()}] {elab} "
        input_3_all += f"Hypothesis: {hyp} "
        for key, elab in hyp_elaborations.items() :
            input_3_all += f"[{key.capitalize()}] {elab} "
        input_3_all += "Is there a contradiction or entailment between the premise and hypothesis ?"
        predictions['S3-all'] = self._prediction_pipeline(input_3_all, self.model_s3_alldims)

        # The input for system 4 is in the same format as for system 1
        predictions['S4'] = self.model_s4.prediction_pipeline(input_1)

        return predictions
    
    '''Uses the predictions from each model to compute the final prediction of the ensemble'''
    def _ensemble_algorithm(self, model_outputs) :
        # Firstly, the label is selected based on the majority between the 5 best models (according to the paper : systems 1, 2, 3-motivation, 3-alldims, 4)
        labels = [model_outputs[key].split('.')[0] for key in ['S1', 'S2', 'S3-mot', 'S3-all', 'S4']]
        # Sometimes, it might happen with the small models that the generated label is a mix of words, like 'Contratailment' or 'Endiction'
        for label in labels :
            if label not in ['Label: Contradiction', 'Label: Entailment'] :
                labels.remove(label)
        unique, counts = np.unique(labels, return_counts=True)
        ix = np.argmax(counts)
        major_label = unique[ix]

        # Then, pick the explanation of the first system agreeing with the majority label, following an order indicated in the paper
        for key in ['S3-cons', 'S3-emo', 'S2', 'S3-all', 'S3-mot', 'S4', 'S1'] :
            substrings = model_outputs[key].split('.')
            label = substrings[0]
            explanation = substrings[1]

            if label == major_label :
                break

        return major_label + '.' + explanation + '.'

    '''Expected input : [Premise_sentence, hypothesis_sentence] or list of inputs'''
    def predict(self, inputs) :
        if isinstance(inputs, list) and all(isinstance(input, str) for input in inputs) and len(inputs) == 2 :
            preds = self._get_all_predictions(inputs)
            final_pred = self._ensemble_algorithm(preds)
        
            return final_pred 
        
        elif isinstance(inputs, list) and all(isinstance(input, list) for input in inputs) :
            predictions = []
            for input in inputs :
                predictions.append(self.predict(input))
            
            return predictions
        else :
            raise TypeError('Inputs should be either a list of two strings or a list of lists of two strings')

In [4]:
operating_modes = [
    ("system_1", preprocess_dataset_s1),
    ("system_2", preprocess_dataset_s2),
    ("system_31", preprocess_dataset_s31),
    ("system_32", preprocess_dataset_s32),
    ("system_33", preprocess_dataset_s33),
    ("system_34", preprocess_dataset_s34),
    ("system_35", preprocess_dataset_s35),
    ("system_4", preprocess_dataset_s1),
    ("system_5", preprocess_dataset_s1),
    ("system_6", preprocess_dataset_s1),
    ("system_7", preprocess_dataset_s7),
    ]

In [5]:
def train_model(name, model, curr_ds):
    training_args = Seq2SeqTrainingArguments(
        output_dir=f"{name}",
        learning_rate=3e-4,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=2*BATCH_SIZE,
        save_total_limit=1,
        num_train_epochs=NUM_EPOCHS,
        report_to="none",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        eval_accumulation_steps=1,
        logging_steps=1,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=curr_ds["train"],
        eval_dataset=curr_ds["val"].select(range(350)),
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    )

    trainer.train()

    return trainer

In [6]:
records = []

In [7]:
from IPython.display import clear_output

modes = {name: [] for name, _ in operating_modes}

for train_idxs, val_idxs in indexes[0:4]:
    fold_dataset = DatasetDict({
        "train": ds.select(train_idxs),
        "val": ds.select(val_idxs)
    })

    for name, preprocess_func in operating_modes:
        curr_ds = fold_dataset.map(preprocess_func, batched=True).remove_columns(fold_dataset['train'].column_names)

        if name == "system_4":
            ds_41 = fold_dataset.map(preprocess_dataset_s41, batched=True).remove_columns(fold_dataset['train'].column_names)
            train_model("system_41", AutoModelForSeq2SeqLM.from_pretrained(base_checkpoint), ds_41)
            ds_42 = fold_dataset.map(preprocess_dataset_s42, batched=True).remove_columns(fold_dataset['train'].column_names)
            train_model("system_42", AutoModelForSeq2SeqLM.from_pretrained(base_checkpoint), ds_42)
        elif name == "system_5":
            pass
        else:
            if name == "system_6":
                model = AutoModelForSeq2SeqLM.from_pretrained("RicoBorra/T5-small-synthetic-FLUTE")
            else:
                model = AutoModelForSeq2SeqLM.from_pretrained(base_checkpoint)
            trainer = train_model(name, model, curr_ds)
            
        # have to do batched rouge computation otherwise not enough memory
        rouge = evaluate.load("rouge")
        metrics = {'rouge1': 0., 'rouge2': 0., 'rougeL': 0., 'rougeLsum': 0.}
        count = 0
        for i in range(0, len(curr_ds['val']), N_GEN):
            count += 1
            if (name != "system_4") and (name != "system_5"):
                (predictions, _), label_ids, _ = trainer.predict(test_dataset=curr_ds['val'].select(range(i, min(i+N_GEN, len(curr_ds['val'])))))
                # delete stuff after EOS token
                predicted_token_ids = torch.argmax(torch.from_numpy(predictions), dim=-1)
                for j in range(predicted_token_ids.shape[0]):
                    ind = (predicted_token_ids[j] == 1).nonzero(as_tuple=True)[0]
                    if ind.numel() != 0:
                        predicted_token_ids[j, ind[0]:] = 1
                decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
                # clean decoded preds if needed
                if name == "system_2":
                    decoded_preds = [dp[dp.find("Label"):] for dp in decoded_preds]
                if name == "system_7":
                    decoded_preds = [ex[ex.find("Label: "):] + " " + ex[:ex.find("Label: ")] for ex in decoded_preds]

            else:
                small_ds = curr_ds['val'].select(range(i, min(i+N_GEN, len(curr_ds['val']))))
                label_ids = small_ds['labels']
                inputs = tokenizer.batch_decode(small_ds['input_ids'], skip_special_tokens=True)
                if name == "system_4":
                    sys_4 = DREAM_FLUTE_System4(tokenizer=None, model_s41_path=get_path("system_41"), model_s42_path=get_path("system_42"))
                    decoded_preds = sys_4.prediction_pipeline(inputs=inputs)
                else: # "name == system_5"
                    inputs = [get_prem_hyp(ex) for ex in inputs]
                    sys_5 = DREAM_FLUTE_Ensemble(
                        tokenizer_path=None, 
                        s1_path=get_path("system_1"),
                        s2_path=get_path("system_2"),
                        s3_emo_path=get_path("system_31"),
                        s3_mot_path=get_path("system_32"),
                        s3_cons_path=get_path("system_33"),
                        s3_rot_path=get_path("system_34"),
                        s3_alldims_path=get_path("system_35"),
                        s4_clas_path=get_path("system_41"),
                        s4_exp_path=get_path("system_42"),
                        dream_path=None)
                    decoded_preds = sys_5.predict(inputs=inputs)
                
            # careful here
            labels = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            if name == "system_2":
                decoded_labels = [dp[dp.find("Label"):] for dp in decoded_labels]
            if name == "system_7":
                decoded_labels = [ex[ex.find("Label: "):] + " " + ex[:ex.find("Label: ")] for ex in decoded_labels]

            new_metrics = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
            for k in new_metrics:
                metrics[k] += new_metrics[k]

        for k in metrics:
                metrics[k] /= count
        
        clear_output(wait=True)
        modes[name].append(metrics['rouge1'])
        print(modes)
        #print(name, decoded_preds[0], decoded_labels[0])
        records.append({'name': name, "content": decoded_preds[0]})
        #print(predicted_token_ids[0], labels[0])

{'system_1': [0.5960144014132844], 'system_2': [0.6013509440969985], 'system_31': [], 'system_32': [], 'system_33': [], 'system_34': [], 'system_35': [], 'system_4': [], 'system_5': [], 'system_6': [], 'system_7': []}
system_2 Label: Entailment. Explanation: It  fun of someone someonea friend is often  and often be very frustratingful and especially if it is to front of stranger that are nots Label: Entailment. Explanation: Being made fun of by a friend is often embarrassing and can be very hurtful, especially if it happens in front of people who are strangers
tensor([ 6632,    10,  9422, 13212,     9, 27111,     5, 16229,    10,   695,
         5756,   297,     5,  1881,  3767,   257,    10,    94,     3,   694,
           13,   841,   841,     9,  1565,    19,   557,     3,    11,   557,
           36,   182, 16591,  1329,    11,   902,     3,    99,    34,    19,
           12,   851,    13, 13037,    24,    33,    59,     7,     1,     1,
            1,     1]) [ 6632    10  9470 1

Map:   0%|          | 0/6780 [00:00<?, ? examples/s]

Map:   0%|          | 0/754 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
records.append({'name': "ground_truth", 'content': decoded_labels[0]})
records

In [ ]:
"""
{'system_1': [0.5056115530372459, 0.5085812472965738], 'system_2': [0.5323239453833644, 0.5374525863206762], 'system_31': [0.5084813364883226, 0.5078069259932709], 'system_32': [0.5055255195638098, 0.5074808302929493], 'system_33': [0.5032615710245999, 0.508591734512484], 'system_34': [0.5067396302748032, 0.5087587260751042], 'system_35': [0.5036829298423671, 0.5073656197676405]}
{'system_1': [0.47329270980754923], 'system_2': [0.5039333868999851], 'system_31': [0.47420432731446144], 'system_32': [0.473849787491779], 'system_33': [0.47287909150934293], 'system_34': [0.47171914165316353], 'system_35': [0.47436950304876985]}
{'system_1': [0.5221458462682274, 0.4906728974226775], 'system_2': [0.5453307495230577, 0.5189864534869755], 'system_31': [0.524029905213074, 0.49432447432649923], 'system_32': [0.5236538569098267, 0.4949640794043393], 'system_33': [0.522696825432352, 0.49418902866839187], 'system_34': [0.5231002650281118, 0.49245360907178504], 'system_35': [0.5238553839735753, 0.4967562420314123]}
{'system_1': [0.5201383622010409, 0.5019083214066056], 'system_2': [0.5451744732719431, 0.5273190341222926], 'system_31': [0.5192974336052382, 0.5002995114825706], 'system_32': [0.5188841013023715, 0.5007719063422055], 'system_33': [0.5203219622545853, 0.49992736417041994], 'system_34': [0.5180675680247785, 0.5008015168679895], 'system_35': [0.5224425242712794, 0.5010754435268072]}
{'system_1': [0.49590791112913984], 'system_2': [0.5147096144113823], 'system_31': [0.4927430972029428], 'system_32': [0.49258799343269694], 'system_33': [0.49049952341283654], 'system_34': [0.49317378483047924], 'system_35': [0.4919880906784929]}
{'system_1': [0.498351050356349, 0.5090844074007058], 'system_2': [0.5289706096196247, 0.535423121872322], 'system_31': [0.5004418699772601, 0.5081924349139884], 'system_32': [0.5000023839289528, 0.5069172466751917], 'system_33': [0.49609349538092806, 0.5060478473679386], 'system_34': [0.49683869684920356, 0.5072891134960673], 'system_35': [0.49873229852851614, 0.5063988604083957]}
"""

"\n{'system_1': [0.5056115530372459, 0.5085812472965738], 'system_2': [0.5323239453833644, 0.5374525863206762], 'system_31': [0.5084813364883226, 0.5078069259932709], 'system_32': [0.5055255195638098, 0.5074808302929493], 'system_33': [0.5032615710245999, 0.508591734512484], 'system_34': [0.5067396302748032, 0.5087587260751042], 'system_35': [0.5036829298423671, 0.5073656197676405]}\n{'system_1': [0.47329270980754923], 'system_2': [0.5039333868999851], 'system_31': [0.47420432731446144], 'system_32': [0.473849787491779], 'system_33': [0.47287909150934293], 'system_34': [0.47171914165316353], 'system_35': [0.47436950304876985]}\n{'system_1': [0.5221458462682274, 0.4906728974226775], 'system_2': [0.5453307495230577, 0.5189864534869755], 'system_31': [0.524029905213074, 0.49432447432649923], 'system_32': [0.5236538569098267, 0.4949640794043393], 'system_33': [0.522696825432352, 0.49418902866839187], 'system_34': [0.5231002650281118, 0.49245360907178504], 'system_35': [0.5238553839735753, 

In [ ]:
inputs[0]

'Premise: I hope that, as neighbours, we may live long in peace and amity, and in the rare exchange of any sort of assistance which should subsist between us. Hypothesis: I hope that, as neighbours, we may live long in peace and amity, and in the interchange of those good offices which should subsist between us. Is there a contradiction or entailment between the premise and hypothesis?'

In [ ]:
modes

{'system_1': [0.498351050356349, 0.5090844074007058],
 'system_2': [0.5289706096196247, 0.535423121872322],
 'system_31': [0.5004418699772601, 0.5081924349139884],
 'system_32': [0.5000023839289528, 0.5069172466751917],
 'system_33': [0.49609349538092806, 0.5060478473679386],
 'system_34': [0.49683869684920356, 0.5072891134960673],
 'system_35': [0.49873229852851614, 0.5063988604083957]}

In [ ]:
l = [{'system_1': [0.5056115530372459, 0.5085812472965738], 'system_2': [0.5323239453833644, 0.5374525863206762], 'system_31': [0.5084813364883226, 0.5078069259932709], 'system_32': [0.5055255195638098, 0.5074808302929493], 'system_33': [0.5032615710245999, 0.508591734512484], 'system_34': [0.5067396302748032, 0.5087587260751042], 'system_35': [0.5036829298423671, 0.5073656197676405]},
{'system_1': [0.47329270980754923], 'system_2': [0.5039333868999851], 'system_31': [0.47420432731446144], 'system_32': [0.473849787491779], 'system_33': [0.47287909150934293], 'system_34': [0.47171914165316353], 'system_35': [0.47436950304876985]},
{'system_1': [0.5221458462682274, 0.4906728974226775], 'system_2': [0.5453307495230577, 0.5189864534869755], 'system_31': [0.524029905213074, 0.49432447432649923], 'system_32': [0.5236538569098267, 0.4949640794043393], 'system_33': [0.522696825432352, 0.49418902866839187], 'system_34': [0.5231002650281118, 0.49245360907178504], 'system_35': [0.5238553839735753, 0.4967562420314123]},
{'system_1': [0.5201383622010409, 0.5019083214066056], 'system_2': [0.5451744732719431, 0.5273190341222926], 'system_31': [0.5192974336052382, 0.5002995114825706], 'system_32': [0.5188841013023715, 0.5007719063422055], 'system_33': [0.5203219622545853, 0.49992736417041994], 'system_34': [0.5180675680247785, 0.5008015168679895], 'system_35': [0.5224425242712794, 0.5010754435268072]},
{'system_1': [0.49590791112913984], 'system_2': [0.5147096144113823], 'system_31': [0.4927430972029428], 'system_32': [0.49258799343269694], 'system_33': [0.49049952341283654], 'system_34': [0.49317378483047924], 'system_35': [0.4919880906784929]},
{'system_1': [0.498351050356349, 0.5090844074007058], 'system_2': [0.5289706096196247, 0.535423121872322], 'system_31': [0.5004418699772601, 0.5081924349139884], 'system_32': [0.5000023839289528, 0.5069172466751917], 'system_33': [0.49609349538092806, 0.5060478473679386], 'system_34': [0.49683869684920356, 0.5072891134960673], 'system_35': [0.49873229852851614, 0.5063988604083957]},
]

In [ ]:
d = {}
for key in l[0].keys():
    d[key] = []
    for el in l:
        d[key] += el[key]


In [ ]:
d

{'system_1': [0.5056115530372459,
  0.5085812472965738,
  0.47329270980754923,
  0.5221458462682274,
  0.4906728974226775,
  0.5201383622010409,
  0.5019083214066056,
  0.49590791112913984,
  0.498351050356349,
  0.5090844074007058],
 'system_2': [0.5323239453833644,
  0.5374525863206762,
  0.5039333868999851,
  0.5453307495230577,
  0.5189864534869755,
  0.5451744732719431,
  0.5273190341222926,
  0.5147096144113823,
  0.5289706096196247,
  0.535423121872322],
 'system_31': [0.5084813364883226,
  0.5078069259932709,
  0.47420432731446144,
  0.524029905213074,
  0.49432447432649923,
  0.5192974336052382,
  0.5002995114825706,
  0.4927430972029428,
  0.5004418699772601,
  0.5081924349139884],
 'system_32': [0.5055255195638098,
  0.5074808302929493,
  0.473849787491779,
  0.5236538569098267,
  0.4949640794043393,
  0.5188841013023715,
  0.5007719063422055,
  0.49258799343269694,
  0.5000023839289528,
  0.5069172466751917],
 'system_33': [0.5032615710245999,
  0.508591734512484,
  0.47287

In [ ]:
from scipy.stats import ttest_rel
# paired t test
for key in list(l[0].keys())[1:]:
    print(key, ttest_rel(d[key], d["system_1"], alternative="greater"))

system_2 Ttest_relResult(statistic=23.19102411438574, pvalue=1.2259514290288666e-09)
system_31 Ttest_relResult(statistic=0.5948201499859518, pvalue=0.2833078171850142)
system_32 Ttest_relResult(statistic=-0.15212858962856382, pvalue=0.5587793758146149)
system_33 Ttest_relResult(statistic=-1.4525349538900347, pvalue=0.9098456440410234)
system_34 Ttest_relResult(statistic=-1.3719083987600567, pvalue=0.898341997218993)
system_35 Ttest_relResult(statistic=0.10660074133722681, pvalue=0.45872204141041606)
